In [1]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='tokuyama'

In [2]:
#モデルスコアデータの読み込み
#score_df = pd.read_csv('../csv/model_score_date_{}.csv'.format(place))#dateアリのモデル
score_df = pd.read_csv('../csv/{}_model_score_normal.csv'.format(place))#dateなしのモデル
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,392840.0,447400.0,395.0,4474.0,87.805096,8.828789,395.0
1,1.0,4.0,52.0,392840.0,447400.0,395.0,4474.0,87.805096,8.828789,395.0
2,1.0,4.0,53.0,392840.0,447400.0,395.0,4474.0,87.805096,8.828789,395.0
3,1.0,4.0,54.0,392840.0,447400.0,395.0,4474.0,87.805096,8.828789,395.0
4,1.0,4.0,55.0,392840.0,447400.0,395.0,4474.0,87.805096,8.828789,395.0
...,...,...,...,...,...,...,...,...,...,...
13855,28.0,8.0,145.0,63140.0,68100.0,21.0,681.0,92.716593,2.349486,16.0
13856,28.0,8.0,146.0,63140.0,68100.0,21.0,681.0,92.716593,2.349486,16.0
13857,28.0,8.0,147.0,54540.0,66500.0,21.0,665.0,82.015038,2.255639,15.0
13858,28.0,8.0,148.0,54540.0,64000.0,21.0,640.0,85.218750,2.343750,15.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [3]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}_date.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除


df=result_df.copy()

#学習、テストデータ切り分け(dateを使って学習データを切り分ける。)

df['date']=pd.to_datetime(df['date'])#文字列なのでdateを日付型に変換
df['year']=df['date'].dt.year
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらdateとyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)
valid_df=valid_df.drop(['date'],axis=1)
train_df=train_df.drop(['date'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])

#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,395,8.828789,994.531646,800.0
1,2,350,7.822977,1070.600000,720.0
2,3,219,4.894949,1482.602740,1010.0
3,4,154,3.442110,2228.051948,1505.0
4,5,225,5.029057,1547.955556,1150.0
...,...,...,...,...,...
113,116,4,0.089405,31752.500000,26600.0
114,117,1,0.022351,25470.000000,25470.0
115,118,1,0.022351,49690.000000,49690.0
116,119,3,0.067054,54600.000000,45120.0


### モデル性能評価機能

In [4]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [5]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
1549,4.0,4.0,115.0,40690.0,30400.0,154.0,304.0,133.848684,11.513158,35.0,64.0
1649,4.0,5.0,116.0,47010.0,40900.0,154.0,409.0,114.938875,9.779951,40.0,65.0
1650,4.0,5.0,117.0,47830.0,39100.0,154.0,391.0,122.327366,10.485934,41.0,66.0
1651,4.0,5.0,118.0,41630.0,33600.0,154.0,336.0,123.898810,11.011905,37.0,67.0
1652,4.0,5.0,119.0,42790.0,30500.0,154.0,305.0,140.295082,12.131148,37.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...
13313,27.0,8.0,98.0,150210.0,132400.0,31.0,1324.0,113.451662,1.737160,23.0,47.0
13314,27.0,8.0,99.0,150210.0,126500.0,31.0,1265.0,118.743083,1.818182,23.0,48.0
13315,27.0,8.0,100.0,150210.0,122200.0,31.0,1222.0,122.921440,1.882160,23.0,49.0
13316,27.0,8.0,101.0,150210.0,122200.0,31.0,1222.0,122.921440,1.882160,23.0,50.0


In [6]:
#model_para_df.to_csv('../csv/{}_good_score_model_date.csv'.format(place), encoding='utf_8_sig')
model_para_df.to_csv('../csv/{}_good_score_model_normal.csv'.format(place), encoding='utf_8_sig')

In [7]:
diff_model

target_com       28.000000
depth             5.000000
target_per      148.000000
総収益           36460.000000
投資金額          40700.000000
出現数              21.000000
購買予測数           407.000000
利益率              89.582310
購買的中率             2.948403
的中数              12.000000
number_i         97.000000
Name: 13561, dtype: float64

In [8]:
len_df

0

In [9]:
type(diff_model)

pandas.core.series.Series